# Bee Recognition Writeup
### Based on Gabriel Preda's notebook "Honey Bee Subspecies Classification"
https://www.kaggle.com/gpreda/honey-bee-subspecies-classification

In [144]:
import pandas as pd
import numpy as np
import sys
import os
import random

In [145]:
import imageio
import skimage
import skimage.io
import skimage.transform
import scipy
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [146]:
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPool2D, Dropout, BatchNormalization,LeakyReLU
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from keras.utils import to_categorical

In [147]:
IMAGE_PATH = 'bee_imgs/'
IMAGE_WIDTH = 100
IMAGE_HEIGHT = 100
IMAGE_CHANNELS = 3
RANDOM_STATE = 2018
TEST_SIZE = 0.1
VAL_SIZE = 0.2
CONV_2D_DIM_1 = 16
CONV_2D_DIM_2 = 16
CONV_2D_DIM_3 = 32
CONV_2D_DIM_4 = 64
MAX_POOL_DIM = 2
KERNEL_SIZE = 3
BATCH_SIZE = 32
NO_EPOCHS_1 = 5
NO_EPOCHS_2 = 10
NO_EPOCHS_3 = 50
PATIENCE = 5
VERBOSE = 1
CLASSES_CHANGE = {1:2, 2:1}

In [148]:
honey_bee_df = pd.read_csv('bee_data.csv')
lst = ['Italian honey bee', 'Russian honey bee', 'Carniolan honey bee']
honey_bee_df = honey_bee_df[honey_bee_df.subspecies.isin(lst)]
honey_bee_df = honey_bee_df.drop(['date', 'time', 'location', 'zip code', 'health', 'pollen_carrying', 'caste'], axis=1)

In [149]:
train_df, test_df = train_test_split(honey_bee_df, test_size=TEST_SIZE, random_state=RANDOM_STATE)
train_df, val_df = train_test_split(train_df, test_size=VAL_SIZE, random_state=RANDOM_STATE)

In [150]:
def read_image(file_name, train=True):
    image = skimage.io.imread(IMAGE_PATH + file_name if train else file_name)
    image = skimage.transform.resize(image, (IMAGE_WIDTH, IMAGE_HEIGHT), mode='reflect')
    return image[:,:,:IMAGE_CHANNELS]

In [151]:
def categories_encoder(dataset, var='subspecies'):
    X = np.stack(dataset['file'].apply(read_image))
    y = pd.get_dummies(dataset[var], drop_first=False)
    return X, y

In [152]:
X_train, y_train = categories_encoder(train_df)
X_val, y_val = categories_encoder(val_df)
X_test, y_test = categories_encoder(test_df)

In [154]:
def create_model():
    model = Sequential()
    model.add(Conv2D(CONV_2D_DIM_1, kernel_size=KERNEL_SIZE, input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT,IMAGE_CHANNELS), activation='relu', padding='same'))
    model.add(MaxPool2D(MAX_POOL_DIM))
    model.add(Conv2D(CONV_2D_DIM_2, kernel_size=KERNEL_SIZE, activation='relu', padding='same'))
    model.add(Flatten())
    model.add(Dense(y_train.columns.size, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [155]:
image_generator = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        rotation_range=180,
        zoom_range = 0.1, 
        width_shift_range=0.1,
        height_shift_range=0.1, 
        horizontal_flip=True,
        vertical_flip=True)

image_generator.fit(X_train)

In [156]:
model = create_model()
train_model1  = model.fit_generator(image_generator.flow(X_train, y_train, batch_size=BATCH_SIZE),
                        epochs=NO_EPOCHS_1,
                        validation_data=[X_val, y_val],
                        steps_per_epoch=len(X_train) / BATCH_SIZE)

Epoch 1/5
91/90 [==============================] - 17s 192ms/step - loss: 0.2584 - acc: 0.9100 - val_loss: 0.1344 - val_acc: 0.9794
Epoch 2/5
91/90 [==============================] - 19s 214ms/step - loss: 0.1033 - acc: 0.9749 - val_loss: 0.0711 - val_acc: 0.9890
Epoch 3/5
91/90 [==============================] - 19s 203ms/step - loss: 0.0529 - acc: 0.9859 - val_loss: 0.0377 - val_acc: 0.9890
Epoch 4/5
91/90 [==============================] - 20s 219ms/step - loss: 0.0403 - acc: 0.9894 - val_loss: 0.0225 - val_acc: 0.9931
Epoch 5/5
91/90 [==============================] - 20s 218ms/step - loss: 0.0298 - acc: 0.9918 - val_loss: 0.0418 - val_acc: 0.9890


In [166]:
from requests import get, post, Session
import json
from bs4 import BeautifulSoup
from PIL import Image
from io import BytesIO

lst = []
session = Session()


page = session.get('http://127.0.0.1:5000/').text
soup = BeautifulSoup(page, 'html.parser')
js = soup.find_all('script')[1]
bee_ids = eval(list(map(lambda x: x.strip(), str(js).split('\n')))[1].split(' = ')[1][:-1])  

In [167]:
classes = {1:2, 2:1, 0:0}

for bid in bee_ids:
    response = get(f'http://127.0.0.1:5000/static/img/{bid}.png')
    image = read_image(BytesIO(response.content), False)
    image = np.expand_dims(image, axis=0)
    
    answer = model.predict_classes(image)
    lst.append(classes[int(answer[0])])
    
      

In [168]:
ans = session.post('http://127.0.0.1:5000/', data = {'answers': str(lst)})
print(ans.text)  

{"accuracy":"100.0%","flag":"paseca{u1tr4_be3_recOgn1T1ooon_m45ter}","success":true}

